## fMOST imaging and processing

As part of the [BRAIN Initiative Cell Census Network (BICCN)](https://biccn.org/) efforts to characterize brain cell types across multiple modalities, [Peng et al 2021](https://www.nature.com/articles/s41586-021-03941-1), established a pipeline to label, image, reconstruct and classify single neurons in mice.

These neurons are labelled by cell subclass or type-selective Cre driver lines, enabling correlation of their morphologies and projection patterns with molecular identities. 

A GFP-labelled brain is first embedded in resin for imaging in the fMOST platform. Following imaging of the entire block face, the top 1 µm of tissue is sliced off with a diamond knife, exposing the next face of the block for imaging. For the entire mouse brain, a 15–20 TB dataset containing about 10,000 coronal planes of 0.2–0.3 µm xy resolution and 1 µm z sampling rate is generated within 2 weeks.

Vaa3D, an open-source, cross-platform visualization and analysis system, to use for reconstructing neuronal morphologies. Further, mBrainAligner software was used to perform 3D registration from fMOST images (subject) to the average template of the Allen CCFv3 (reference). Axonal and dendritic morphological features and projection patterns were quantified and used to identifed 11 major projection neuron types.

The [fMOST imaging data](https://download.brainimagelibrary.org/biccn/zeng/luo/fMOST/) and [1,741 neuronal reconstructions (original and mapped)](https://doi.brainimagelibrary.org/doi/10.35077/g.25) are available for download at the Brain Imaging Library (BIL)

[Extended Data Fig.3](https://www.nature.com/articles/s41586-021-03941-1/figures/7): Platform and workflow of the brain-wide complete morphology imaging, reconstruction, registration and analysis pipeline.
![Platform and Workflow](ExtendFig3.png)

## Allen Common Coordinate Framework CCFv3

Allen Mouse Brain Common Coordinate Framework (CCFv3, [Wang et al, 2020](https://doi.org/10.1016/j.cell.2020.04.007)) is a 3D reference space by creating an
average brain at 10um voxel resolution from serial two-photon tomography images of 1,675 young adult C57Bl6/J mice. Using multimodal reference data, we parcellated the entire brain directly in 3D, labeling every voxel with a brain structure spanning 43 isocortical areas and their layers, 314 subcortical gray matter structures, 81 fiber tracts, and 8 ventricular structures.

CCFv is used in informatics pipelines and online applications to analyze, visualize and integrate multimodal and multiscale data sets in 3D, and is openly accessible for research use. CCFv3 is currently been used as the integration framework for the projects such as [Allen Brain Map](https://portal.brain-map.org/), [BICCN](https://biccn.org/), [IBL](https://www.internationalbrainlab.com/) and [eBrains](https://ebrains.eu/). 

![AllenCCFv3](AllenCCFv3.png)

## Registration Challenge

This aim of this hackathon is to generate reproducible pipelines to register whole-brain microscopy image data to the CCFv3. The fMOST dataset described above is particular challenging due to the distortion caused by the fixation and dehydration specimen processing steps.

A typical first step in a registraton workflow is to downsample the imaging to preform the initial registration. For the hackathon, we have pre-downsample to 7-10 µm resolution volumes. The downsampled data can be found in this directory:
https://download.brainlib.org/hackathon/2022_GYBS/input/fMOST/subject/ which can be downloaded or directly accessed via one of the provided hackathon workstations.

The volumes are in [NIfTI](https://nifti.nimh.nih.gov/) file format which is commonly used in neuroimaging and can be opened in Python using software such as [SimpleITK](https://simpleitk.org/) and [NiBabel](https://nipy.org/nibabel/nifti_images.html) or in viewers such as [Slicer](https://www.slicer.org/) and [ITKSNAP](http://www.itksnap.org/pmwiki/pmwiki.php)

In the following section, we will demonstrate how to read the reference and one of the subject volume using SimpleITK. Refer to the [SimpleITK documentation](https://simpleitk.readthedocs.io/en/master/gettingStarted.html) for more information of features and usage.

In [1]:
# Install notebook dependencies
import sys

#!{sys.executable} -m pip install SimpleITK

In [2]:
# Import notebook dependencies
import os
import requests
import shutil

import SimpleITK as sitk
import matplotlib.pyplot as plt

In [3]:
# Retrieve data from download.brainlib.org
files = ['fMOST/reference/average_template_25_mm_ASL.nii.gz','fMOST/subject/194061_red_mm_SLA.nii.gz']

os.makedirs("fMOST", exist_ok=True)
os.makedirs("fMOST/reference", exist_ok=True)
os.makedirs("fMOST/subject", exist_ok=True)

for f in files :
    if os.path.exists(f):
        print(f"{f} already exists.")
    else:
        download_url = f"https://download.brainlib.org/hackathon/2022_GYBS/input/{f}"
        print(f"Downloading {download_url} into {f}")
        response = requests.get(download_url, stream=True)
        with open(f, "wb") as fp:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, fp)


fMOST/reference/average_template_25_mm_ASL.nii.gz already exists.
fMOST/subject/194061_red_mm_SLA.nii.gz already exists.


In [4]:
# Read images from files
reference = sitk.ReadImage( files[0] )
subject = sitk.ReadImage( files[1] )

In [5]:
# Function to print out image information
def image_info( img ) :
    print('size: ' + str(img.GetSize()) + ' voxels')
    print('spacing: ' + str(img.GetSpacing()) + ' mm' )
    print('direction: ' + str(img.GetDirection()) )
    print('origin: ' + str(img.GetOrigin()))

In SimpleITK, the orientation is encoded as direction cosine matrix 
with respect to **RAI** configuration (x: right->left, y: anterior->posterior, z: inferior(ventral)->superior(dorsal).

The reference volume:
- is 25µm isotropic resolution
- has size of 528x320x456 voxels
- has orientation  **ASL** (x: anterior->posterior, y: superior->inferior, z: left->right)
- has origin located in the RAI corner of the volume

In [6]:
# Print out reference volume image information
image_info( reference )

size: (528, 320, 456) voxels
spacing: (0.02500000037252903, 0.02500000037252903, 0.02500000037252903) mm
direction: (-0.0, 0.0, -1.0, 1.0, -0.0, 0.0, 0.0, -1.0, 0.0)
origin: (11.375, 0.0, 7.974999904632568)


The subject volume :
- is 10µm isotropic resolution
- has size of 591x1029x1076) voxels
- has orientation **SLA** (x: superior->inferior, y: left->right, z: anterior->posterior)
- has origin located in the RAI corner of the volume

In [7]:
# Print out subject volume image information
image_info( subject )

size: (591, 1029, 1076) voxels
spacing: (0.009999999776482582, 0.009999999776482582, 0.009999999776482582) mm
direction: (-0.0, -1.0, 0.0, 0.0, -0.0, 1.0, -1.0, 0.0, 0.0)
origin: (10.279999732971191, 0.0, 5.900000095367432)


## Observations

Neuroimaging applications like ITKSNAP and Slicer will use these parameters and reslice and display images slices in world coordinates so that volumes of different orientation and spacing can be visually compared. 

Comparing the reference (CCFv3) and subject (one fMOST dataset) you can observed that after fixation and dehydration, the specimen is 20% smaller than typical brain and the size of the ventricles appears relatively larger. Additionally, intensity profile between the reference and subject volumes are significantly different with subject volume have much lower anatomical differentiation.

![Reference and subject in ITKSNAP](ITKSNAP_axial_reference_subject.png)

Additional artifacts includes bright white strips in the background on the left side of the subject brain, as well as regular intensity striping along the horizontal axis.

![Coronal slice of subject in ITKSNAP](ITKSNAP_coronal_subject.png)
![Sagittal slice of subject in ITKSNAP](ITKSNAP_sagittal_subject.png)

Looking at mid-sagittal section of the subject volume provides some insight at the type of distortion that has occured and is reminiscent of the configuration of the brain during development.

![Midsagittal slice of subject in ITKSNAP](ITKSNAP_midsagittal_reference.png)
![Midsagittal slice of subject in ITKSNAP](ITKSNAP_midsagittal_subject.png)
![E15.5 reference atlas](DevMouse_reference_atlas.png)